# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [40]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'webdriver_manager'

In [22]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [23]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = 'https://static.bc-edx.com/data/web/mars_facts/index.html'
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [24]:
# Create a Beautiful Soup Object
html = browser.html
html_soup = soup(html, 'html.parser')

In [25]:
# Extract all rows of data
weather = html_soup.find_all('tr', class_='data-row')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [26]:
# Create an empty list
weather_data = []
# Loop through the scraped data to create a list of rows
for row in weather:
    # Find all the cells in the row
    cells = row.find_all('td')
    # Extract the data from the cells
    id = cells[0].text
    terrestrial_date = cells[1].text
    sol = cells[2].text
    ls = cells[3].text
    month = cells[4].text
    min_temp = cells[5].text
    pressure = cells[6].text
    
    # Store in a ditctionary
    weather_stats = {
        'id': id,
        'terrestrial_date': terrestrial_date,
        'sol': sol,
        'ls': ls,
        'month': month,
        'min_temp': min_temp,
        'pressure': pressure
    }
    # Add the dictionary to the list
    weather_data.append(weather_stats)

In [27]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
columns = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
df = pd.DataFrame(weather_data, columns = columns)

In [28]:
# Confirm DataFrame was created successfully
df.head()

,id,terrestrial_date,sol,ls,month,min_temp,pressure


### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [29]:
# Examine data type of each column
df.dtypes

id                  object
terrestrial_date    object
sol                 object
ls                  object
month               object
min_temp            object
pressure            object
dtype: object

In [30]:
# Change data types for data analysis
# Change data types for data analysis
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date'])
df[['sol', 'ls', 'month']]= df[['sol', 'ls', 'month']].astype('int64')
df[['min_temp', 'pressure']]= df[['min_temp', 'pressure']].astype('float64')

In [31]:
# Confirm type changes were successful by examining data types again
df.dtypes

id                          object
terrestrial_date    datetime64[ns]
sol                          int64
ls                           int64
month                        int64
min_temp                   float64
pressure                   float64
dtype: object

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [32]:
# 1. How many months are there on Mars?
df['month'].value_counts().sort_index()

Series([], Name: month, dtype: int64)

In [33]:
# 2. How many Martian days' worth of data are there?
df['id'].nunique()

0

In [34]:
# 3. What is the average low temperature by month?
avg_low_temp = df.groupby(['month']).mean()['min_temp']
avg_low_temp

Series([], Name: min_temp, dtype: float64)

In [38]:
# Plot the average temperature by month
avg_low_temp.plot(kind = 'bar', figsize = (15,10))
plt.xlabel('Month', fontsize = 20)
plt.ylabel('Temperature in Celsius', fontsize = 20)
plt.xticks(rotation = 0, fontsize = 20)
plt.yticks(fontsize = 20)
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
cold_to_hot = avg_low_temp.sort_values(ascending = True)
cold_to_hot.plot(kind = 'bar', figsize = (15,10))
plt.xlabel('Month', fontsize = 20)
plt.ylabel('Temperature in Celsius', fontsize = 20)
plt.xticks(rotation = 0, fontsize = 20)
plt.yticks(fontsize = 20)
plt.show()

In [37]:
# 4. Average pressure by Martian month
avg_pressure = df.groupby(['month']).mean()['pressure']
avg_pressure

Series([], Name: pressure, dtype: float64)

In [ ]:
# Plot the average pressure by month
sort_avg_pressure = avg_pressure.sort_values(ascending = True)
sort_avg_pressure.plot(kind = 'bar', figsize = (15,10))
plt.xlabel('Month', fontsize = 20)
plt.ylabel('Atmospheric Pressure', fontsize = 20)
plt.xticks(rotation = 0, fontsize = 20)
plt.yticks(fontsize = 20)
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
df.plot(y = 'min_temp', use_index = True, legend = False, figsize = (15,10))
plt.xlabel('Number of Terrestrial Days', fontsize = 20)
plt.ylabel('Minimum Temperature', fontsize = 20)
plt.xticks(rotation = 0, fontsize = 20)
plt.yticks(fontsize = 20)
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
df.to_csv('output/mars_weather', index = False)

In [ ]:
browser.quit()